# Set up environment

In [ ]:
import pandas as pd
import pandas_market_calendars as mcal
import torch
import matplotlib.pyplot as plt
import numpy as np
import random
from IPython.display import display
from gymnasium import spaces
from mmd.train import get_robustq_params_dicts, training_info
from mmd.evaluation import simulate_agent_spx
from agent.q import QFunc
from agent.DQN import PORDQN
from functools import reduce

total_length = 560
burn_in = 500
state_len = 60
cal_start_date = '1995-01-01'
cal_end_date = '2023-12-31'
trading_calendar = 'NYSE'
calendar = mcal.get_calendar(trading_calendar)
schedule = calendar.schedule(start_date=cal_start_date, end_date=cal_end_date)

int_rate = 0.024
trans_cost = 0.0005 # standard cost = 0.0005
eval_batch_size = 1000
eval_seed = 12345

batch_size = 8
device = 'cpu'
action_space = spaces.Discrete(9)
action_values = torch.linspace(-1., 1., 9, device=device)
num_actions = len(action_values)
nu_dist = 't'
nu_scale = 0.03
nu_df = 2
other_state_vars = ['log_wealth', 'positions', 'dt']
obs_dim = state_len + len(other_state_vars)

# RUN IF TRAINING FROM SCRATCH
discount = 0.99
eps_greedy = 0.1 # epsilon greedy parameter
buffer_max_length = int(1e5)
clone_steps = 50
train_steps = 1
agent_batch_size = 128
n_batches = 1
n_epochs = 1
robustq_lr = 1e-4
architecture = [64, 64]
pre_train_Q = False
n_episodes = 3

robustq = QFunc(state_len+len(other_state_vars), architecture, action_values.shape[0]).to(device)

delta = 1e-4 # regularisation parameter for Sinkhorn distance
epsilon = 0.003 # Sinkhorn distance
norm_ord = 1
lamda_init = 0. # initial lambda
lamda_max_iter = 100
lamda_step_size = 10 # step size for learning rate scheduler
lamda_gamma = 0 # gamma for learning rate scheduler
lamda_lr = [0.04,0.2,0.3,0.5,0.8,1.5,3,100,1000,10000,100000]
n_outer = 1 # not used in this algorithm but used in logging by writer
n_inner = 1000 # number of samples from nu to calc inner expectations


writer = None
seed = None
simulator_params, model_params = get_robustq_params_dicts(vars().copy())

In [37]:
def highlight_numeric_groups(df):
    styles = pd.DataFrame("", index=df.index, columns=df.columns)
    
    num_cols = df.select_dtypes(include=[np.number]).columns
    non_num_cols = df.columns.difference(num_cols)

    for col in num_cols:
        if col in ["Volatility", "Down_deviation"]:
            min_val = df[col].max()
            max_val = df[col].min()
        else:
            max_val = df[col].max()
            min_val = df[col].min()

        styles[col] = np.where(
            df[col] == max_val, "background-color: darkgreen; color: white;",
            np.where(df[col] == min_val, "background-color: darkred; color: white;", "")
        )

    styles = styles[list(num_cols) + list(non_num_cols)]

    return styles


def highlight_numeric_groups_std(df):
    styles = pd.DataFrame("", index=df.index, columns=df.columns)
    num_cols = df.select_dtypes(include=[np.number]).columns
    non_num_cols = df.columns.difference(num_cols)

    for col in num_cols:
        min_val = df[col].max()
        max_val = df[col].min()
        styles[col] = np.where(
            df[col] == max_val, "background-color: darkgreen; color: white;",
            np.where(df[col] == min_val, "background-color: darkred; color: white;", "")
        )
    styles = styles[list(num_cols) + list(non_num_cols)]
    return styles

results_detail = {0.05: {}, 0.25: {}}

spx = pd.read_pickle(f"stats/0.05_2.5").loc["spx"]
spx = round(spx, 3)
spx = pd.DataFrame(spx).T
spx.index = ["S&P 500"]
spx["ε and δ"] = "-"
cols_to_front = ["ε and δ"]
spx_cols = ["Final_wealth", "Max_drawdown", "Sharpe", "Sortino", "Down_deviation", "Volatility"]
spx = spx[cols_to_front + [col for col in spx_cols]]

mean_summary = {}
std_summary = {}

for txn_cost in (0.05, 0.25):
    file = f"{txn_cost}_3_og"
    og = pd.read_pickle(f"stats/{file}").loc[0:]
    results_detail[txn_cost]["3_og"] = og

    og_mean = og.mean()
    og_std = og.std()
    og_mean["Epsilon"], og_std["Epsilon"] = "OG (3e-3)", "OG (3e-3)"
    og_mean["Delta"], og_std["Delta"] = "1e-4", "1e-4"
    rows = []
    stdev = []

    file = f"{txn_cost}_3_delta"
    run = pd.read_pickle(f"stats/{file}").loc[0:]
    results_detail[txn_cost]["delta"] = run

    delta_mean = run.mean()
    delta_std = run.std()
    delta_mean["Epsilon"], delta_std["Epsilon"] = "3e-3", "3e-3"
    delta_mean["Delta"], delta_std["Delta"] = "1e-5", "1e-5"

    for eps in (2.5, 3, 3.5):
        if eps == 3:
            rows.append(og_mean)
            stdev.append(og_std)
        
        if eps == 3.5:
            rows.append(delta_mean)
            stdev.append(delta_std)
        
        file = f"{txn_cost}_{eps}"
        run = pd.read_pickle(f"stats/{file}").loc[0:]
        results_detail[txn_cost][eps] = run

        mean = run.mean()
        std = run.std()
        eps = str(eps) + "e-3"
        mean["Epsilon"], std["Epsilon"] = eps, eps
        mean["Delta"], std["Delta"] = "1e-4", "1e-4"
        rows.append(mean)
        stdev.append(std)

    mean_df = pd.DataFrame(rows)
    mean_summary[txn_cost] = mean_df
    std_df = pd.DataFrame(stdev)
    std_summary[txn_cost] = std_df

# Define the desired column order
col_order = ["Epsilon", "Delta", "Final_wealth", "Max_drawdown", "Sharpe", "Sortino", "Down_deviation", "Volatility"]
num_cols = mean_df.select_dtypes(include=[np.number]).columns

# Reorder columns for all mean and std DataFrames
for txn_cost in mean_summary:
    mean_summary[txn_cost] = mean_summary[txn_cost][col_order]
    std_summary[txn_cost] = std_summary[txn_cost][col_order]


means = {}
for txn_cost, df in mean_summary.items():
    df.index = df.index.map(str)
    means[txn_cost] = df.style.apply(highlight_numeric_groups, axis=None).format("{:.3f}", subset=num_cols).hide(axis="index")
()
stds = {}
for txn_cost, df in std_summary.items():
    df.index = df.index.map(str)
    stds[txn_cost] = df.style.apply(highlight_numeric_groups_std, axis=None).format("{:.3f}", subset=num_cols).hide(axis="index")


means[0.00] = spx

In [42]:
print("----------------------------------- Mean Values ---------------------------------------")
display(means[0.00])
print("------------------------------- 0.05% Transaction Cost --------------------------------")
display(means[0.05])
print("------------------------------- 0.25% Transaction Cost --------------------------------")
display(means[0.25])

----------------------------------- Mean Values ---------------------------------------


,ε and δ,Final_wealth,Max_drawdown,Sharpe,Sortino,Down_deviation,Volatility
S&P 500,-,9.524,-0.568,0.41,0.569,0.138,0.191


------------------------------- 0.05% Transaction Cost --------------------------------


Epsilon,Delta,Final_wealth,Max_drawdown,Sharpe,Sortino,Down_deviation,Volatility
2.5e-3,1e-4,2.290,-0.401,0.195,0.272,0.091,0.125
OG (3e-3),1e-4,2.103,-0.364,0.210,0.292,0.081,0.113
3e-3,1e-4,2.566,-0.415,0.251,0.347,0.096,0.132
3e-3,1e-5,1.833,-0.361,0.190,0.264,0.073,0.100
3.5e-3,1e-4,1.926,-0.386,0.192,0.267,0.079,0.109


------------------------------- 0.25% Transaction Cost --------------------------------


Epsilon,Delta,Final_wealth,Max_drawdown,Sharpe,Sortino,Down_deviation,Volatility
2.5e-3,1e-4,2.711,-0.355,0.275,0.387,0.084,0.118
OG (3e-3),1e-4,2.416,-0.300,0.303,0.421,0.071,0.098
3e-3,1e-4,2.680,-0.292,0.323,0.454,0.072,0.100
3e-3,1e-5,2.196,-0.401,0.237,0.330,0.081,0.113
3.5e-3,1e-4,2.916,-0.339,0.296,0.407,0.082,0.113


In [ ]:
print("------------------------------- 0.05% Transaction Cost -------------------------------")
display(stds[0.05])
print("------------------------------- 0.25% Transaction Cost -------------------------------")
display(stds[0.25])

------------------------------- 0.05% Transaction Cost -------------------------------


Epsilon,Delta,Final_wealth,Max_drawdown,Sharpe,Sortino,Down_deviation,Volatility
2.5e-3,1e-4,0.891,0.097,0.141,0.195,0.014,0.021
OG (3e-3),1e-4,0.538,0.020,0.076,0.108,0.011,0.017
3e-3,1e-4,0.115,0.068,0.035,0.053,0.012,0.015
3e-3,1e-5,0.652,0.054,0.129,0.182,0.008,0.011
3.5e-3,1e-4,0.674,0.130,0.125,0.176,0.012,0.018


------------------------------- 0.25% Transaction Cost -------------------------------


Epsilon,Delta,Final_wealth,Max_drawdown,Sharpe,Sortino,Down_deviation,Volatility
2.5e-3,1e-4,0.914,0.060,0.077,0.113,0.015,0.020
OG (3e-3),1e-4,0.695,0.029,0.076,0.113,0.013,0.019
3e-3,1e-4,0.839,0.060,0.077,0.117,0.019,0.028
3e-3,1e-5,0.698,0.147,0.115,0.161,0.020,0.026
3.5e-3,1e-4,1.233,0.050,0.136,0.186,0.020,0.028


In [66]:
#  To inspect each seed for any configuration, choose from transaction cost and configuration

txn_cost = 0.05
# txn_cost = 0.25

keys = [2.5, "3_og", 3, "delta", 3.5]
# Select key from configurations above
key = keys[1]

print(key, ", transaction cost: ", txn_cost)
results_detail[txn_cost][key]

3_og , transaction cost:  0.05


,Final_wealth,Volatility,Sharpe,Sortino,Down_deviation,Max_drawdown
0,2.455795,0.122841,0.254643,0.354130,0.088331,-0.331683
1,2.027843,0.118641,0.207468,0.288456,0.085331,-0.360888
2,2.273810,0.121022,0.236321,0.332616,0.085985,-0.386227
3,1.209194,0.081796,0.080853,0.107485,0.061529,-0.368371
4,2.550381,0.119780,0.272135,0.377954,0.086244,-0.373685
